In [1]:
!pip3 install --upgrade pip
!pip install numpy pandas
!pip install -U "psycopg[binary]"

In [2]:
import psycopg, os
import sqlite3
from datetime import datetime
import pandas as pd

conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="my_database",
    user="postgres",
    password="yangyuhao0314120") #change your password

/var/folders/3p/wh0d3zx97gggmnqj8m47cmbw0000gn/T/ipykernel_7683/781056724.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
cur = conn.cursor()

In [16]:
createCmd = """
CREATE TABLE hosts (
    host_id INT PRIMARY KEY,
    host_name VARCHAR(255)
);

CREATE TABLE neighbourhood_groups (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) UNIQUE
);

CREATE TABLE neighbourhoods (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    neighbourhood_group_id INT,
    FOREIGN KEY (neighbourhood_group_id) REFERENCES neighbourhood_groups(id)
);

CREATE TABLE accommodations (
    id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    neighbourhood_id INT,
    latitude DECIMAL(9,6),
    longitude DECIMAL(9,6),
    room_type VARCHAR(50),
    price INT,
    minimum_nights INT,
    number_of_reviews INT,
    last_review DATE,
    reviews_per_month DECIMAL(4,2),
    calculated_host_listings_count INT,
    availability_365 INT,
    number_of_reviews_ltm INT,
    FOREIGN KEY (host_id) REFERENCES hosts(host_id),
    FOREIGN KEY (neighbourhood_id) REFERENCES neighbourhoods(id)
);



CREATE TABLE airports (
    airport_code VARCHAR(3) PRIMARY KEY,
    start_city VARCHAR(225)
);

  
CREATE TABLE airport_to_neighborhood_by_subway (
    airport_name VARCHAR(3),
    neighborhood_id INT,
    hours VARCHAR(10),
    price FLOAT,
    carbon FLOAT,
    PRIMARY KEY (airport_name, neighborhood_id),
    FOREIGN KEY (airport_name) REFERENCES airports(airport_code),
    FOREIGN KEY (neighborhood_id) REFERENCES neighbourhoods(id)
);
    
CREATE TABLE airport_to_neighborhood_by_car (
    airport_name VARCHAR(3),
    neighborhood_id INT,
    hours VARCHAR(10),
    price FLOAT,
    carbon FLOAT,
    PRIMARY KEY (airport_name, neighborhood_id),
    FOREIGN KEY (airport_name) REFERENCES airports(airport_code),
    FOREIGN KEY (neighborhood_id) REFERENCES neighbourhoods(id)
);

 
CREATE TABLE Flights (
    legId VARCHAR(255) PRIMARY KEY,
    searchDate DATE,
    flightDate DATE,
    startingAirport VARCHAR(3),
    destinationAirport VARCHAR(3),
    fareBasisCode VARCHAR(255),
    travelDuration VARCHAR(10),
    elapsedDays INT,
    isBasicEconomy BOOLEAN,
    isRefundable BOOLEAN,
    isNonStop BOOLEAN,
    baseFare DECIMAL(10, 2),
    totalFare DECIMAL(10, 2),
    seatsRemaining INT,
    totalTravelDistance DECIMAL(10, 2),
    FOREIGN KEY (startingAirport) REFERENCES airports(airport_code),
    FOREIGN KEY (destinationAirport) REFERENCES airports(airport_code)
);


CREATE TABLE AircraftModels (
    modelId SERIAL PRIMARY KEY,
    modelName VARCHAR(255),
    planeModelName VARCHAR(255),
    engineType VARCHAR(255),
    fuelCapacity VARCHAR(255),
    emptyWeightLbs INT,
    lengthFtIn VARCHAR(255),
    wingSpanFtIn VARCHAR(255),
    rangeNM INT,
    price INT
);

CREATE TABLE EngineDetails (
    engineId SERIAL PRIMARY KEY,
    modelId INT,
    engineType VARCHAR(255),
    hpOrLbsThrustEachEngine VARCHAR(255),
    FOREIGN KEY (modelId) REFERENCES AircraftModels(modelId)
);

CREATE TABLE PerformanceMetrics (
    performanceId SERIAL PRIMARY KEY,
    modelId INT,
    maxSpeedKnots VARCHAR(10),
    rcmndCruiseKnots INT,
    stallKnotsDirty INT,
    allEngineRateOfClimb VARCHAR(255),
    engOutRateOfClimb VARCHAR(255),
    takeoffOver50ft VARCHAR(255),
    landingOver50ft VARCHAR(255),
    FOREIGN KEY (modelId) REFERENCES AircraftModels(modelId)
)
"""
cur.execute(createCmd)
conn.commit()

DuplicateTable: relation "hosts" already exists

In [4]:
import pandas as pd

# Read the CSV files
df1 = pd.read_csv('/Users/yuhaoyang/Downloads/itineraries.csv')
df2= pd.read_csv('/Users/yuhaoyang/Downloads/Airbnb.csv')
df3 = pd.read_csv('/Users/yuhaoyang/Downloads/Plane Price.csv')
df4 = pd.read_csv('/Users/yuhaoyang/Downloads/airports_to_neighbourhoods.csv')
df5 = pd.read_csv('/Users/yuhaoyang/Downloads/hosts.csv')
df6 = pd.read_csv('/Users/yuhaoyang/Downloads/neighborhoods.csv')
df7 = pd.read_csv('/Users/yuhaoyang/Downloads/airports.csv')

# Drop rows with NaN values and assign the cleaned DataFrames to new variables
cleaned_df1 = df1.dropna()
cleaned_df2 = df2.dropna()
cleaned_df3 = df3.dropna()

# Reset the index for cleaned_df3
cleaned_df1 = cleaned_df1.reset_index(drop=True)
cleaned_df2 = cleaned_df2.reset_index(drop=True)
cleaned_df3 = cleaned_df3.reset_index(drop=True)


In [5]:
def process_row(row):
    processed_row = []
    for item in row:
        processed_row.append(item)
    return processed_row

# Apply the function to each row
processed_data1 = cleaned_df1.apply(lambda row: process_row(row), axis=1)
processed_data2 = cleaned_df2.apply(lambda row: process_row(row), axis=1)
processed_data3 = cleaned_df3.apply(lambda row: process_row(row), axis=1)
processed_data4 = df4.apply(lambda row: process_row(row), axis=1)
processed_data5 = df5.apply(lambda row: process_row(row), axis=1)
processed_data6 = df6.apply(lambda row: process_row(row), axis=1)
processed_data7 = df7.apply(lambda row: process_row(row), axis=1)

In [6]:
processed_data1

0        [e66e999b88c18d9bfe059e3fcfdbe000, 4/16/2022, ...
1        [11f28cd9ca7f8d4c9abb36aca47b21a1, 4/16/2022, ...
2        [dc1f39f8d6b3c9c3ff322c596d82ae43, 4/16/2022, ...
3        [425588319fe49adcdb945f9b884f9099, 4/16/2022, ...
4        [315e98eefb6b4fde04b2eae04af909df, 4/16/2022, ...
                               ...                        
76740    [e4b06fdc519d05856bb6f626b00b6254, 4/18/2022, ...
76741    [68063b72dbacaee5c40af1cf1974faff, 4/18/2022, ...
76742    [e4c75b5085f239097321d53baf03c297, 4/18/2022, ...
76743    [dcc6b40d6c5cc190e7546a8fa076d189, 4/18/2022, ...
76744    [ee1aeb9b67208606b0f38aa6866715db, 4/18/2022, ...
Length: 76745, dtype: object

In [7]:
processed_data2[15]

[6990.0,
 'UES Beautiful Blue Room',
 16800,
 'Cyn',
 'Manhattan',
 'East Harlem',
 40.78778,
 -73.94759,
 'Private room',
 62,
 30,
 243,
 '2023/1/1',
 1.49,
 1,
 249,
 7,
 10]

In [34]:
processed_data4

0      [Midtown, LGA, 22min, 40min, 70, 2.9, 3.588, 0...
1      [Bedford-Stuyvesant, LGA, 35min, 1hr14min, 55,...
2      [Upper West Side, LGA, 34min, 1hr3min, 63, 2.9...
3      [Sunset Park, LGA, 48min, 1hr17min, 80, 2.9, 5...
4      [Clinton Hill, LGA, 29min, 1hr16min, 48, 2.9, ...
                             ...                        
218    [Lighthouse Hill, LGA, 1hr7min, 2hr13min, 135,...
219    [Fort Wadsworth, LGA, 58min, 1hr51min, 117, 2....
220    [New Dorp, LGA, 1hr10min, 1hr53min, 135, 2.9, ...
221    [Neponsit, LGA, 59min, 1hr56min, 111, 2.9, 8.9...
222    [Willowbrook, LGA, 1hr3min, 1hr58min, 128, 2.9...
Length: 223, dtype: object

In [7]:
insert_statement = """
INSERT INTO AircraftModels (modelId, modelName, planeModelName, engineType, fuelCapacity, emptyWeightLbs, lengthFtIn, wingSpanFtIn, rangeNM, price) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
serial_numbers = range(1, len(processed_data3) + 1)

# Insert records into the table using only the desired columns
for i, row in enumerate(processed_data3):
    serial_number = str(serial_numbers[i])  # Convert serial number to string
    data_tuple = (serial_number, row[0], row[0], row[1], row[6], row[11], row[12], row[13], row[14], row[15]) 
    cur.execute(insert_statement, data_tuple)

UniqueViolation: duplicate key value violates unique constraint "aircraftmodels_pkey"
DETAIL:  Key (modelid)=(1) already exists.

In [10]:
insert_statement = """
INSERT INTO hosts (host_id, host_name) 
VALUES (%s, %s)
"""
for row in processed_data5:
    data_tuple = (row[0], row[1]) 
    cur.execute(insert_statement, data_tuple)

UniqueViolation: duplicate key value violates unique constraint "hosts_pkey"
DETAIL:  Key (host_id)=(2845) already exists.

In [40]:
cur.execute("INSERT INTO neighbourhood_groups VALUES ('1','Manhattan')")
cur.execute("INSERT INTO neighbourhood_groups VALUES ('2','Brooklyn')")
cur.execute("INSERT INTO neighbourhood_groups VALUES ('3','Queens')")
cur.execute("INSERT INTO neighbourhood_groups VALUES ('4','Bronx')")
cur.execute("INSERT INTO neighbourhood_groups VALUES ('5','Staten Island')")

UniqueViolation: duplicate key value violates unique constraint "neighbourhood_groups_pkey"
DETAIL:  Key (id)=(1) already exists.

In [13]:
insert_statement = """
INSERT INTO neighbourhoods (id, name, neighbourhood_group_id) 
VALUES (%s, %s, %s)
"""
for row in processed_data6:
    data_tuple = (row[1], row[0], row[2]) 
    cur.execute(insert_statement, data_tuple)

In [12]:
insert_statement = """
INSERT INTO accommodations (id, name, host_id, neighbourhood_id, latitude, longitude, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365, number_of_reviews_ltm) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
serial_numbers = range(1, len(processed_data2) + 1)

for i, row in enumerate(processed_data2):
    serial_number = str(serial_numbers[i])  # Convert serial number to string
    data_tuple = (serial_number, row[1], row[2], row[17], row[6], row[7], row[8],row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16]) 
    cur.execute(insert_statement, data_tuple)

In [16]:
insert_statement = """
INSERT INTO airports (airport_code, start_city) 
VALUES (%s, %s)
"""
for row in processed_data7:
    data_tuple = (row[0], row[1]) 
    cur.execute(insert_statement, data_tuple)

In [9]:
insert_statement = """
INSERT INTO airport_to_neighborhood_by_subway (airport_name, neighborhood_id, hours, price, carbon) 
VALUES (%s, %s, %s, %s, %s)
"""
for row in processed_data4:
    data_tuple = (row[1], row[9], row[3], row[5],row[7]) 
    cur.execute(insert_statement, data_tuple)

In [10]:
insert_statement = """
INSERT INTO airport_to_neighborhood_by_car (airport_name, neighborhood_id, hours, price, carbon) 
VALUES (%s, %s, %s, %s, %s)
"""
for row in processed_data4:
    data_tuple = (row[1], row[9], row[2], row[4],row[6]) 
    cur.execute(insert_statement, data_tuple)

In [14]:
insert_statement = """
INSERT INTO Flights (legId, searchDate, flightDate, startingAirport, destinationAirport, fareBasisCode, travelDuration, elapsedDays, isBasicEconomy, isRefundable, isNonStop, baseFare, totalFare, seatsRemaining, totalTravelDistance) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
serial_numbers = range(1, len(processed_data1) + 1)

for i, row in enumerate(processed_data1):
    serial_number = str(serial_numbers[i])
    data_tuple = (serial_number, row[1], row[2], row[3],row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14]) 
    cur.execute(insert_statement, data_tuple)

In [17]:
insert_statement = """
INSERT INTO EngineDetails (engineId, modelId, engineType, hpOrLbsThrustEachEngine) 
VALUES (%s, %s, %s, %s)
"""
serial_numbers = range(1, len(processed_data3) + 1)

for i, row in enumerate(processed_data3):
    serial_number = str(serial_numbers[i])
    data_tuple = (serial_number, serial_number, row[1], row[2]) 
    cur.execute(insert_statement, data_tuple)

In [19]:
insert_statement = """
INSERT INTO PerformanceMetrics (performanceId, modelId, maxSpeedKnots, rcmndCruiseKnots, stallKnotsDirty, allEngineRateOfClimb, engOutRateOfClimb, takeoffOver50ft, landingOver50ft) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
serial_numbers = range(1, len(processed_data3) + 1)

for i, row in enumerate(processed_data3):
    serial_number = str(serial_numbers[i])
    data_tuple = (serial_number, serial_number, row[3], row[4], row[5], row[7], row[8], row[9], row[10]) 
    cur.execute(insert_statement, data_tuple)

In [30]:
Flight_Price_Trend_Analysis= """
SELECT
    startingAirport,
    destinationAirport,
    EXTRACT(YEAR FROM flightDate) AS flightYear,
    EXTRACT(MONTH FROM flightDate) AS flightMonth,
    AVG(totalFare) AS avgPrice
FROM
    Flights
GROUP BY
    startingAirport,
    destinationAirport,
    EXTRACT(YEAR FROM flightDate),
    EXTRACT(MONTH FROM flightDate)
ORDER BY
    startingAirport,
    destinationAirport,
    flightYear,
    flightMonth;
"""

cur.execute(Flight_Price_Trend_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('ATL', 'LGA', Decimal('2022'), Decimal('4'), Decimal('296.2728000000000000'))
('ATL', 'LGA', Decimal('2022'), Decimal('5'), Decimal('189.7352802570510532'))
('ATL', 'LGA', Decimal('2022'), Decimal('6'), Decimal('214.9861702127659574'))
('BOS', 'LGA', Decimal('2022'), Decimal('4'), Decimal('247.5626550270921132'))
('BOS', 'LGA', Decimal('2022'), Decimal('5'), Decimal('120.9756384790011351'))
('BOS', 'LGA', Decimal('2022'), Decimal('6'), Decimal('127.3936637931034483'))
('CLT', 'LGA', Decimal('2022'), Decimal('4'), Decimal('278.0023724616146607'))
('CLT', 'LGA', Decimal('2022'), Decimal('5'), Decimal('163.7783741935483871'))
('CLT', 'LGA', Decimal('2022'), Decimal('6'), Decimal('178.3922975352112676'))
('DEN', 'LGA', Decimal('2022'), Decimal('4'), Decimal('293.1388385502471170'))
('DEN', 'LGA', Decimal('2022'), Decimal('5'), Decimal('257.3131937911571025'))
('DEN', 'LGA', Decimal('2022'), Decimal('6'), Decimal('311.1327142857142857'))
('DFW', 'LGA', Decimal('2022'), Decimal('4'), Decima

In [4]:
Hotel_Occupancy_Rate_Analysis="""
SELECT
    a.name AS hotelName,  -- Specify table alias for 'name' column from Accommodations
    COUNT(*) AS numBookings
FROM
    Accommodations a
JOIN
    neighbourhoods n ON a.neighbourhood_id = n.id  -- Join the tables with aliases
GROUP BY
    a.name  -- Group by 'name' from Accommodations
ORDER BY
    hotelName;
"""
cur.execute(Hotel_Occupancy_Rate_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('!! Large 3 Bedroom Apartment in NYC!!', 1)
('!!! Designer room in NYC!!! 12 min to Time Square', 1)
('!!Beautiful Large 3 Bedroom Apartment in NYC!!', 1)
('!!Charming 3bedroom apartment on upper west side!!', 1)
('!!Studio next to Empire State Bldg.', 1)
('!Beautiful renovated apartment¡ two bedrooms, Plus', 1)
('!Spacious & Modern Brooklyn Loft-Long Term Stays!', 1)
('" Bedroom  in the Nearest  Place from Manhattan."', 1)
('" EXTRA LARGE BED ROOM AT A PRICE YOU CAN AFFORD"', 1)
('" Modern Studio Apartment', 1)
('"1 Sweet Suite" VERYFAIR $ Brand New Queens Studio', 1)
('"2 Bedroom highrise with Stunning views of NY/NJ"', 1)
('"4th best neighborhood in the world" - Time Out', 1)
('"ADORABLE CENTRAL 1BD LITTLE ITALY/SOHO"', 1)
('"ADORABLE DOWNTOWN APT CENTRAL TO ALL �- NYC"', 1)
('"BEST PLACE TO LIVE" in NYC, North Park Slope', 1)
('"BQE Rental" 2 BR Chic Suite in Spring Creek', 1)
('"Beautiful Traveler Room" Brooklyn Medical Student', 1)
('"Beautiful place, 15 to 20 minutes from Manha

In [4]:
Neighborhood_Booking_Analysis="""
SELECT
    n.name,
    COUNT(*) AS numBookings
FROM
    Accommodations a
JOIN
    neighbourhoods n ON a.neighbourhood_id = n.id
GROUP BY
    n.name;
"""

cur.execute(Neighborhood_Booking_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Tremont', 25)
('Parkchester', 39)
('Brooklyn Heights', 70)
('Highbridge', 23)
('Eltingville', 3)
('Castleton Corners', 5)
('Marble Hill', 8)
('Fieldston', 7)
('Richmondtown', 2)
('Longwood', 62)
('Gerritsen Beach', 2)
('Flatlands', 107)
('Port Morris', 53)
('Kew Gardens Hills', 29)
('Cypress Hills', 188)
('SoHo', 199)
('Edenwald', 25)
('Oakwood', 7)
('Columbia St', 25)
('Bellerose', 17)
('Norwood', 30)
('Lower East Side', 494)
('Glendale', 55)
('Little Italy', 86)
('Bergen Beach', 21)
('Eastchester', 13)
('Cambria Heights', 53)
('Rego Park', 73)
('NoHo', 28)
('East New York', 319)
('Bayside', 52)
('Rossville', 3)
('Randall Manor', 14)
('Morningside Heights', 138)
('Bay Terrace', 5)
('Jackson Heights', 188)
('Navy Yard', 7)
('Sea Gate', 17)
('Astoria', 632)
('New Brighton', 9)
('Flushing', 415)
('Morrisania', 15)
('Silver Lake', 3)
('East Elmhurst', 258)
('Inwood', 143)
('Bensonhurst', 60)
('Park Slope', 273)
('Chelsea, Staten Island', 2)
('Concourse', 50)
('Middle Village', 27)
('Man

In [10]:
Neighborhood_Group_Booking_Analysis="""
SELECT
    ng.name AS neighborhoodGroup,
    COUNT(*) AS numBookings
FROM
    Accommodations a
JOIN
    neighbourhoods n ON a.neighbourhood_id = n.id
JOIN
    neighbourhood_groups ng ON n.neighbourhood_group_id = ng.id
GROUP BY
    ng.name;
"""

cur.execute(Neighborhood_Group_Booking_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Brooklyn', 12888)
('Bronx', 1358)
('Manhattan', 12513)
('Queens', 5497)
('Staten Island', 363)


In [11]:
Room_Type_Analysis="""
SELECT
    n.name AS neighborhood,
    a.room_type,
    COUNT(*) AS numBookings
FROM
    Accommodations a
JOIN
    neighbourhoods n ON a.neighbourhood_id = n.id
GROUP BY
    n.name,
    a.room_type
ORDER BY
    n.name,
    numBookings DESC;
"""

cur.execute(Room_Type_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Allerton', 'Entire home/apt', 27)
('Allerton', 'Private room', 18)
('Arden Heights', 'Entire home/apt', 3)
('Arrochar', 'Entire home/apt', 8)
('Arrochar', 'Private room', 5)
('Arverne', 'Entire home/apt', 79)
('Arverne', 'Private room', 28)
('Astoria', 'Private room', 351)
('Astoria', 'Entire home/apt', 271)
('Astoria', 'Shared room', 9)
('Astoria', 'Hotel room', 1)
('Bath Beach', 'Entire home/apt', 19)
('Bath Beach', 'Private room', 6)
('Battery Park City', 'Entire home/apt', 37)
('Battery Park City', 'Private room', 9)
('Bay Ridge', 'Private room', 61)
('Bay Ridge', 'Entire home/apt', 51)
('Bay Ridge', 'Shared room', 6)
('Bay Terrace', 'Entire home/apt', 3)
('Bay Terrace', 'Private room', 2)
('Baychester', 'Entire home/apt', 16)
('Baychester', 'Private room', 12)
('Bayside', 'Entire home/apt', 29)
('Bayside', 'Private room', 23)
('Bayswater', 'Private room', 10)
('Bayswater', 'Entire home/apt', 9)
('Bedford-Stuyvesant', 'Entire home/apt', 1318)
('Bedford-Stuyvesant', 'Private room'

In [14]:
Neighborhood_Group_Room_Type_Analysis="""
SELECT
    ng.name AS neighborhoodGroup,
    a.room_type,
    COUNT(*) AS numBookings
FROM
    Accommodations a
JOIN
    neighbourhoods n ON a.neighbourhood_id = n.id
JOIN
    neighbourhood_groups ng ON n.neighbourhood_group_id = ng.id
GROUP BY
    ng.name,
    a.room_type
ORDER BY
    ng.name,
    numBookings DESC;

"""

cur.execute(Neighborhood_Group_Room_Type_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Bronx', 'Private room', 684)
('Bronx', 'Entire home/apt', 644)
('Bronx', 'Shared room', 30)
('Brooklyn', 'Entire home/apt', 7544)
('Brooklyn', 'Private room', 5206)
('Brooklyn', 'Shared room', 131)
('Brooklyn', 'Hotel room', 7)
('Manhattan', 'Entire home/apt', 7990)
('Manhattan', 'Private room', 4225)
('Manhattan', 'Shared room', 157)
('Manhattan', 'Hotel room', 141)
('Queens', 'Private room', 2752)
('Queens', 'Entire home/apt', 2643)
('Queens', 'Shared room', 94)
('Queens', 'Hotel room', 8)
('Staten Island', 'Entire home/apt', 218)
('Staten Island', 'Private room', 145)


In [7]:
Airport_Neighbourhoods_by_Car_Price_Analysis="""
SELECT
    n.name AS neighborhood,
    AVG(a.price) AS avgPrice
FROM
    airport_to_neighborhood_by_car a
JOIN
    neighbourhoods n ON a.neighborhood_id = n.id
GROUP BY
    n.name;
"""

cur.execute(Airport_Neighbourhoods_by_Car_Price_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Tremont', 57.0)
('Parkchester', 57.0)
('Brooklyn Heights', 55.0)
('Highbridge', 52.0)
('Eltingville', 149.0)
('Castleton Corners', 117.0)
('Marble Hill', 59.0)
('Fieldston', 75.0)
('Richmondtown', 131.0)
('Longwood', 48.0)
('Gerritsen Beach', 98.0)
('Flatlands', 95.0)
('Port Morris', 36.0)
('Kew Gardens Hills', 34.0)
('Cypress Hills', 53.0)
('SoHo', 56.0)
('Edenwald', 66.0)
('Oakwood', 140.0)
('Columbia St', 59.0)
('Bellerose', 59.0)
('Norwood', 60.0)
('Lower East Side', 47.0)
('Glendale', 48.0)
('Little Italy', 53.0)
('Bergen Beach', 96.0)
('Eastchester', 80.0)
('Cambria Heights', 61.0)
('Rego Park', 29.0)
('NoHo', 54.0)
('East New York', 62.0)
('Bayside', 45.0)
('Rossville', 148.0)
('Randall Manor', 125.0)
('Morningside Heights', 55.0)
('Bay Terrace', 40.0)
('Jackson Heights', 17.0)
('Navy Yard', 45.0)
('Sea Gate', 115.0)
('Astoria', 25.0)
('New Brighton', 129.0)
('Flushing', 27.0)
('Morrisania', 55.0)
('Silver Lake', 118.0)
('East Elmhurst', 15.0)
('Inwood', 73.0)
('Bensonhurst', 

In [8]:
Carbon_Difference="""
SELECT
    a.airport_name,
    n.name AS neighborhood,
    (a.carbon - s.carbon) AS carbonOutputDifference
FROM
    airport_to_neighborhood_by_car a
JOIN
    airport_to_neighborhood_by_subway s ON a.airport_name = s.airport_name AND a.neighborhood_id = s.neighborhood_id
JOIN
    neighbourhoods n ON a.neighborhood_id = n.id
ORDER BY
    carbonOutputDifference DESC;
"""

cur.execute(Carbon_Difference)

rows = cur.fetchall()
for row in rows:
    print(row)

('LGA', 'Tottenville', 13.689)
('LGA', "Prince's Bay", 13.65)
('LGA', 'Woodrow', 12.597)
('LGA', 'Huguenot', 12.48)
('LGA', 'Rossville', 12.207)
('LGA', 'Eltingville', 11.232)
('LGA', 'Arden Heights', 11.076)
('LGA', 'Chelsea, Staten Island', 10.647)
('LGA', 'Clifton', 10.335)
('LGA', 'Great Kills', 10.335)
('LGA', 'Mariners Harbor', 10.296)
('LGA', 'New Springville', 10.179)
('LGA', 'Graniteville', 9.828)
('LGA', 'Lighthouse Hill', 9.789)
('LGA', 'Richmondtown', 9.75)
('LGA', 'Oakwood', 9.633)
('LGA', 'Sea Gate', 9.633)
('LGA', 'New Brighton', 9.399)
('LGA', 'New Dorp', 9.243)
('LGA', 'New Dorp Beach', 9.243)
('LGA', 'Willowbrook', 9.204)
('LGA', 'Westerleigh', 9.165)
('LGA', 'Randall Manor', 9.048)
('LGA', 'Todt Hill', 9.048)
('LGA', 'Breezy Point', 8.97)
('LGA', 'West Brighton', 8.97)
('LGA', 'Neponsit', 8.931)
('LGA', 'Grant City', 8.814)
('LGA', 'Belle Harbor', 8.775)
('LGA', 'Midland Beach', 8.775)
('LGA', 'Howland Hook', 8.775)
('LGA', 'Gravesend', 8.775)
('LGA', 'Castleton Corn

In [9]:
Customer_Segmentation_Analysis="""
SELECT
    CASE
        WHEN a.price < 100 THEN 'Budget'
        WHEN a.price >= 100 AND a.price < 200 THEN 'Standard'
        ELSE 'Luxury'
    END AS customerSegment,
    COUNT(*) AS numBookings
FROM
    Accommodations a
GROUP BY
    customerSegment;
"""

cur.execute(Customer_Segmentation_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('Luxury', 7843)
('Budget', 12363)
('Standard', 12413)


In [5]:
Plane_Price_Analysis="""
WITH EngineMaxPrices AS (
    SELECT
        engineType,
        MAX(price) AS maxPrice
    FROM
        AircraftModels
    GROUP BY
        engineType
)
SELECT
    em.modelName AS mostExpensiveModel,
    em.price AS maxPrice,
    em.engineType
FROM
    AircraftModels em
JOIN
    EngineMaxPrices maxp ON em.engineType = maxp.engineType AND em.price = maxp.maxPrice;
"""

cur.execute(Plane_Price_Analysis)

rows = cur.fetchall()
for row in rows:
    print(row)

('A,B 33 Debonair', 3220000, 'Piston')
('100 Darter (S.L. Industries)', 3220000, 'Piston')
('T 337 G-P II,H-P, Skymaster pressurized', 5100000, 'Jet')
('100 Darter (S.L. Industries)', 5100000, 'Jet')
('T 210 K,L Turbo (K=9 mph less speed)', 5100000, 'Jet')
('100 Darter (S.L. Industries)', 1350000, 'piston')
('100 Darter (S.L. Industries)', 3584000, 'Propjet')
